In [ ]:
# Finally sucess full result got

In [7]:
import rasterio
import shapefile
import json
from shapely.geometry import shape, Polygon
import base64

# Paths to the image and shapefile
image_path = "D:/E/Delet/yolo/13_ID42.tif"
shapefile_path = "D:/E/Delet/yolo/13_ID42_combined.shp"

# Read the image to get its dimensions and transformation
with rasterio.open(image_path) as src:
    width, height = src.width, src.height
    transform = src.transform
    image_data = src.read(1)

# Read the shapefile
sf = shapefile.Reader(shapefile_path)

# Initialize a list to store the shapes
shapes = []

# Function to convert coordinates to pixel row and column
def coords_to_pixel(x, y, transform):
    col, row = ~transform * (x, y)
    return col, row

# Loop through each shape in the shapefile
for shape_record in sf.shapeRecords():
    label = shape_record.record['Tree_sp']  # Assuming the label is stored in the attribute table
    geom = shape(shape_record.shape.__geo_interface__)
    
    if isinstance(geom, Polygon):
        points = list(geom.exterior.coords)
    else:
        points = []

    # Convert coordinates to pixel row and column
    pixel_points = [coords_to_pixel(x, y, transform) for x, y in points]

    # Create the shape
    shape_data = {
        "label": label,
        "points": [[float(col), float(row)] for col, row in pixel_points],
        "group_id": None,
        "description": "",
        "shape_type": "polygon",
        "flags": {},
        "mask": None
    }

    shapes.append(shape_data)

# Read and encode the image data
with open(image_path, "rb") as image_file:
    encoded_image_data = base64.b64encode(image_file.read()).decode('utf-8')

# Create the JSON data
json_data = {
    "version": "5.5.0",
    "flags": {},
    "shapes": shapes,
    "imagePath": image_path.split('/')[-1].replace('.tif', '.jpg'),
    "imageData": encoded_image_data,
    "imageHeight": height,
    "imageWidth": width
}

# Save the JSON data to a file
output_path = "D:/E/Delet/yolo/13_ID42.json"
with open(output_path, 'w') as f:
    json.dump(json_data, f, indent=4)

print(f"LabelMe format data saved to {output_path}")


LabelMe format data saved to D:/E/Delet/yolo/13_ID42.json


In [ ]:
#labelme2yolo --json_dir /path/to/labelme_json_dir/